# Demo crawl data từ web Amazon.com

In [1]:
import pandas as pd
import matplotlib as plt
from bs4 import BeautifulSoup
import requests
from requests_html import HTMLSession
import pandas as pd
import os
import pickle

#### Các hàm để tìm kiếm các đặc trưng từ các thẻ trên trang web

In [2]:
#Lấy đặc trưng công từ web
def get_company(soup):
    company = ""
    try:
        company = soup.find("tr", attrs={"class": 'a-spacing-small po-brand'}).find("span", attrs={"class": 'a-size-base po-break-word'}).text.strip()

    except AttributeError:

        company = ""

    return company

#Lấy đặc trưng tên laptop từ web
def get_name(soup):
    name = ""
    try:
        name = soup.find("span", attrs={"id": 'productTitle'}).text.strip()

    except AttributeError:

        name = ""

    return name

#Lấy đặc trưng kích thước màn hình từ web
def get_inches(soup):
    inches = ""
    try:
        inches = soup.find("tr", attrs={"class": 'a-spacing-small po-display.size'}).find("span", attrs={"class": 'a-size-base po-break-word'}).text.strip()

    except AttributeError:

        try:
            
            table = soup.find('table', {'id': 'productDetails_techSpec_section_1'})
            for tr in table.find_all('tr'):
                th = tr.find('th')
                td = tr.find('td')
                if th and th.text.strip() == 'Standing screen display size' and td:
                    inches = td.text.strip()

        except:
            inches = ""

    return inches

#Lấy đặc trưng độ phân giải từ web
def get_screen_resolution(soup):
    screen_resolution = ""
    try:
        screen_resolution = ""
        table = soup.find('table', {'id': 'productDetails_techSpec_section_1'})

        for tr in table.find_all('tr'):
            th = tr.find('th')
            td = tr.find('td')
            if th and th.text.strip() == 'Max Screen Resolution' and td:
                screen_resolution = td.text.strip()
                

    except AttributeError:

        screen_resolution = ""

    return screen_resolution

#Lấy đặc trưng cpu từ web
def get_cpu(soup):
    cpu = ""
    try:
        
        cpu = soup.find("tr", attrs={"class": 'a-spacing-small po-cpu_model.family'}).find("span", attrs={"class": 'a-size-base po-break-word'}).text.strip()        

    except AttributeError:
        try:
            
            table = soup.find('table', {'id': 'productDetails_techSpec_section_1'})
            for tr in table.find_all('tr'):
                th = tr.find('th')
                td = tr.find('td')
                if th and th.text.strip() == 'Processor' and td:
                    cpu = td.text.strip()

        except:
            cpu = ""

    return cpu

#Lấy đặc trưng ram từ web
def get_ram(soup):
    ram = ""
    try:
        
        ram = soup.find("tr", attrs={"class": 'a-spacing-small po-ram_memory.installed_size'}).find("span", attrs={"class": 'a-size-base po-break-word'}).text.strip()      

    except AttributeError:
        try:
            
            table = soup.find('table', {'id': 'productDetails_techSpec_section_1'})
            for tr in table.find_all('tr'):
                th = tr.find('th')
                td = tr.find('td')
                if th and th.text.strip() == 'RAM' and td:
                    ram = td.text.strip()

        except:
            ram = ""

    return ram

#Lấy đặc trưng memory từ web
def get_memory(soup):
    memory = ""
    try:
        
        memory = soup.find("tr", attrs={"class": 'a-spacing-small po-hard_disk.size'}).find("span", attrs={"class": 'a-size-base po-break-word'}).text.strip()      

    except AttributeError:
        try:
            
            table = soup.find('table', {'id': 'productDetails_techSpec_section_1'})
            for tr in table.find_all('tr'):
                th = tr.find('th')
                td = tr.find('td')
                if th and th.text.strip() == 'Hard Drive' and td:
                    memory = td.text.strip()

        except:
            memory = ""

    return memory

#Lấy đặc trưng gpu từ web
def get_gpu(soup):
    gpu = ""
    try:
        
        table = soup.find('table', {'id': 'productDetails_techSpec_section_1'})
        for tr in table.find_all('tr'):
            th = tr.find('th')
            td = tr.find('td')
            if th and th.text.strip() == 'Graphics Coprocessor' and td:
                gpu = td.text.strip()

    except AttributeError:
        gpu = ""

    return gpu

#Lấy đặc trưng hệ điều hành từ web
def get_opsys(soup):
    opsys = ""
    try:
        
        table = soup.find('table', {'id': 'productDetails_techSpec_section_2'})
        for tr in table.find_all('tr'):
            th = tr.find('th')
            td = tr.find('td')
            if th and th.text.strip() == 'Operating System' and td:
                opsys = td.text.strip()

    except AttributeError:
        opsys = ""

    return opsys

#Lấy đặc trưng cân nặng từ web
def get_weights(soup):
    weights = ""
    try:
        
        table = soup.find('table', {'id': 'productDetails_techSpec_section_2'})
        for tr in table.find_all('tr'):
            th = tr.find('th')
            td = tr.find('td')
            if th and th.text.strip() == 'Item Weight' and td:
                weights = td.text.strip()

    except AttributeError:
        weights = ""

    return weights

#Lấy đặc trưng giá từ web
def get_price(soup):
    price = ""
    try:
        price = soup.find("span", attrs={"class": 'a-price aok-align-center reinventPricePriceToPayMargin priceToPay'}).find("span", attrs={"class": 'a-offscreen'}).text.strip()
        
    except AttributeError:
        try:
            price = soup.find("div", attrs={"id": 'corePrice_feature_div'}).find("span", attrs={"class": 'a-offscreen'}).text.strip()

        except:
            try:
                price = new_soup.find('table', {'id': 'HLCXComparisonTable'}).find('tr', {'id': 'comparison_price_row'}).find('td', {'class': 'comparison_baseitem_column'}).find('span', {'class': 'a-color-price a-text-bold'}).text.strip()
            except:
                try:
                    price = new_soup.find('table', {'id': 'HLCXComparisonTable'}).find('tr', {'id': 'comparison_price_row'}).find('td', {'class': 'comparison_baseitem_column'}).find('span', {'class': 'a-offscreen'}).text.strip()
                except:
                    price=""
    return price

##### Hàm tạo kết nối đến web và lấy url các page

In [3]:
def getdata(url):
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
    webpage = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "html.parser")
    return soup

def get_next(soup):
    page = soup.find('span', {'class':'s-pagination-strip'})
    if not page.find('span',{'class': 's-pagination-item s-pagination-next s-pagination-disabled'}):
        url = 'https://www.amazon.com' + str(page.find('a',{'class':'s-pagination-item s-pagination-next s-pagination-button s-pagination-separator'}).get('href'))
        return url
    else:
        return

In [11]:
# mở file lấy tất cả url của mỗi laptop
with open("demo_object_urls.pickle", "rb") as f:
    all_object_links = pickle.load(f)
len(all_object_links)

431

##### Crawl 10 mẫu data từ web

In [12]:
d = {"Company":[], "Name":[], "Inches":[], "Screen_Resolution":[],"CPU":[], "RAM":[],"Memory":[],"GPU":[],"OpSys":[],"Weights":[], "Price":[]}
all_object_links_temp = all_object_links[1:20]

for link in all_object_links_temp:
        new_soup = getdata(link)
        d['Company'].append(get_company(new_soup))
        d['Name'].append(get_name(new_soup))
        d['Inches'].append(get_inches(new_soup))
        d['Screen_Resolution'].append(get_screen_resolution(new_soup))
        d['CPU'].append(get_cpu(new_soup))
        d['RAM'].append(get_ram(new_soup))
        d['Memory'].append(get_memory(new_soup))
        d['GPU'].append(get_gpu(new_soup))
        d['OpSys'].append(get_opsys(new_soup))
        d['Weights'].append(get_weights(new_soup))
        d['Price'].append(get_price(new_soup))

In [15]:
amazon_df = pd.DataFrame.from_dict(d)
amazon_df.head(10)

,Company,Name,Inches,Screen_Resolution,CPU,RAM,Memory,GPU,OpSys,Weights,Price
0,Acer,"Acer Aspire Vero AV15-51-7617 Green PC | 15.6""...",15.6 Inches,‎1920 x 1080,Intel Core i7,16 GB,‎512 SSD,,‎Windows 11 Home,‎3.96 pounds,$509.99
1,ASUS,ASUS Vivobook Go 15 L510 Thin & Light Laptop C...,15.6 Inches,‎1920 x 1080 Pixels,Celeron,4 GB,64 GB,,‎Windows 11 S,‎3.46 pounds,$189.99
2,Acer,"Acer Aspire 5 A515-56-347N Slim Laptop - 15.6""...",15.6 Inches,‎1920x1080,Core I3 1115G4,8 GB,128 GB,‎Intel UHD Graphics,‎Windows 11 S,‎3.64 pounds,$299.99
3,Dell,"Dell Newest Inspiron 15 3511 Laptop, 15.6"" FHD...",15.6 Inches,‎1920 x 1080 Pixels,Core i5 Family,32 GB,1 TB,‎Intel UHD Graphics,‎Windows 11,‎3.74 pounds,$625.00
4,HP,"Newest HP 14"" HD Laptop, Windows 11, Intel Cel...",15.6 Inches,‎1366 x 768 Pixels,Celeron,4 GB,64 GB,‎Graphics_coprocessor,‎Windows 11,‎4.55 pounds,$216.99
5,SAMSUNG,"SAMSUNG 2022 14” FHD(1920 x 1080) Laptop, Wind...",14 Inches,‎1920x1080 Pixels,Snapdragon,4 GB,64 GB,‎Qualcomm,‎Windows 11,‎3.4 pounds,$165.00
6,HP,"HP 2021 Stream 14"" HD SVA Laptop Computer, Int...",14 Inches,‎1366 x 768 Pixels,Celeron N,4 GB,64 GB,‎Intel UHD Graphics 600,‎Windows 10 S,‎3.17 pounds,From $289.00
7,Acer,"Acer Aspire Vero AV15-51-7617 Green PC | 15.6""...",15.6 Inches,‎1920 x 1080,Intel Core i7,16 GB,‎512 SSD,,‎Windows 11 Home,‎3.96 pounds,$509.99
8,Acer,"Acer Aspire 5 A515-56-347N Slim Laptop - 15.6""...",15.6 Inches,‎1920x1080,Core I3 1115G4,8 GB,128 GB,‎Intel UHD Graphics,‎Windows 11 S,‎3.64 pounds,$299.99
9,Lenovo,"Lenovo 2022 Newest Ideapad 3 Laptop, 15.6"" HD ...",15.6 Inches,‎1366 x 768 Pixels,Intel Core i3,8 GB,256 GB,‎Intel UHD Graphics,‎Windows 11 Home,‎3.74 pounds,$379.99


In [16]:
# Lưu raw data vào file csv
amazon_df.to_csv("amazon_laptop_data.csv", header=True, index=False)

##### Tìm kiếm tất cả các url page

In [4]:
url = "https://www.amazon.com/s?k=laptop&ref=nb_sb_noss"
all_page_urls = []
soup = getdata(url)
while True:
    all_page_urls.append(url)
    next_url = get_next(soup)
    if next_url:
        url = next_url
        soup = getdata(url)
    else:
        break



In [5]:
print(all_page_urls)

['https://www.amazon.com/s?k=laptop&ref=nb_sb_noss', 'https://www.amazon.com/s?k=laptop&page=2&qid=1683795594&ref=sr_pg_1', 'https://www.amazon.com/s?k=laptop&page=3&qid=1683795596&ref=sr_pg_2', 'https://www.amazon.com/s?k=laptop&page=4&qid=1683795598&ref=sr_pg_3', 'https://www.amazon.com/s?k=laptop&page=5&qid=1683795601&ref=sr_pg_4', 'https://www.amazon.com/s?k=laptop&page=6&qid=1683795603&ref=sr_pg_5', 'https://www.amazon.com/s?k=laptop&page=7&qid=1683795605&ref=sr_pg_6', 'https://www.amazon.com/s?k=laptop&page=8&qid=1683795606&ref=sr_pg_7', 'https://www.amazon.com/s?k=laptop&page=9&qid=1683795608&ref=sr_pg_8', 'https://www.amazon.com/s?k=laptop&page=10&qid=1683795610&ref=sr_pg_9', 'https://www.amazon.com/s?k=laptop&page=11&qid=1683795612&ref=sr_pg_10', 'https://www.amazon.com/s?k=laptop&page=12&qid=1683795614&ref=sr_pg_11', 'https://www.amazon.com/s?k=laptop&page=13&qid=1683795615&ref=sr_pg_12', 'https://www.amazon.com/s?k=laptop&page=14&qid=1683795617&ref=sr_pg_13', 'https://www.am

In [6]:
# Lưu url page vào file
with open("demo_page_urls.pickle", "wb") as f:
    pickle.dump(all_page_urls, f)

##### Tìm kiếm tất cả các url của laptop từ mỗi page

In [7]:
def get_object_links(soup):
    object_links = []
    objects = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal' })
    for obj in objects:
        object_links.append('https://www.amazon.com' + obj.get('href'))
    return object_links

all_object_links = []
for page_url in all_page_urls:
    soup = getdata(page_url)
    object_links = get_object_links(soup)
    all_object_links.extend(object_links)

len(all_object_links)

431

In [8]:
# Lưu url object vào file
with open("demo_object_urls.pickle", "wb") as f:
    pickle.dump(all_object_links, f)


In [9]:
print(all_object_links[17])

https://www.amazon.com/HP-15-Laptop-Processor-Super-Fast/dp/B0BQ9W4DV3/ref=sr_1_13?keywords=laptop&qid=1683795687&sr=8-13
